In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import timeit

import cupy as cp
cp.random.seed(12)

#### Portions of this were borrowed and adapted from the
#### cuDF cheatsheet, existing cuDF documentation,
#### and 10 Minutes to Pandas.

In [ ]:
import time

import cupy as cp
import cudf
from cuml.neighbors import NearestNeighbors
from cuml.common.device_selection import using_device_type, set_global_device_type, get_global_device_type

# import dask_cudf
# from dask.distributed import Client, wait
# from dask_cuda import LocalCUDACluster



In [ ]:
# cluster = LocalCUDACluster(n_workers=1, threads_per_worker=512)
# client = Client(cluster)

In [ ]:
# Generate a "coordinate" dataframe
n_points = 400000
#df = cudf.DataFrame()

point_cloud = torch.cuda.FloatTensor(n_points, 3).uniform_()

# df['x'] = point_cloud[:, 0]
# df['y'] = point_cloud[:, 1]
# df['z'] = point_cloud[:, 2]

# Generate random data for the DataFrame
random_data = np.array(point_cloud.cpu())

# Create the cudf.DataFrame
df = cudf.DataFrame(random_data, columns=["X", "Y", "Z"])

In [ ]:
# ddf = dask_cudf.from_cudf(df, npartitions=1)
# ddf.head()

# ddf = ddf.persist()

In [ ]:
n_neighbors = 16

In [ ]:
from cuml.datasets import make_blobs
from time import perf_counter

# # Generate random data for the DataFrame
# random_data2 = np.random.rand(n_points, 3)

# # Create the cudf.DataFrame
# fit_df = cudf.DataFrame(random_data2, columns=["X", "Y", "Z"])

# fit model
model = NearestNeighbors(n_neighbors=n_neighbors)
model.fit(df)

# start_time = perf_counter()

# # get 3 nearest neighbors
# distances_fit, indices_fit = model.kneighbors(fit_df)

# end_time = perf_counter()
# print(f"Average execution time with blobs: {end_time-start_time:.6f} seconds")


start_time = perf_counter()

# get 3 nearest neighbors
distances1, indices1 = model.kneighbors(df)

end_time = perf_counter()
print(f"Average execution time with common data: {end_time-start_time:.6f} seconds")

In [ ]:
# fit_df.shape == df.shape

In [ ]:

def autoSearchAlgorithm(df_input):
    # Create a cuML NearestNeighbors model
    nn = NearestNeighbors(n_neighbors=n_neighbors)

    # Compute the Dask DataFrame
    # ddf_computed = ddf.compute()

    # Fit the model with the input data
    nn.fit(df_input)

    start_time = timeit.default_timer()

    # Get the nearest neighbors
    distances2, indices2 = nn.kneighbors(df_input)

    end_time = timeit.default_timer()
    execution_time = end_time - start_time
    print(f"auto Execution time: {execution_time} seconds")
    # print(indices.compute())

    return distances2, indices2, execution_time


In [ ]:
distances2, indices2, execution_time_auto = autoSearchAlgorithm(df)

In [ ]:
indices1.head() == indices2.head()

In [ ]:
def rbcSearchAlgorithm(df_input):
    # Create a cuML NearestNeighbors model
    nn2 = NearestNeighbors(n_neighbors=n_neighbors, algorithm='rbc')

    # Fit the model with the input data
    nn2.fit(df_input)

    start_time = timeit.default_timer()

    # Get the nearest neighbors
    distances3, indices3 = nn2.kneighbors(df_input)

    end_time = timeit.default_timer()
    execution_time = end_time - start_time
    print(f"rbc Execution time: {execution_time} seconds")

    return distances3, indices3, execution_time

distances3, indices3, execution_time_rbc = rbcSearchAlgorithm(df)


In [ ]:
indices2.head() == indices3.head()

In [ ]:
def ivfflatSearchAlgorithm(df_input):
    # Create a cuML NearestNeighbors model
    nn3 = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ivfflat')

    # Fit the model with the input data
    nn3.fit(df_input)

    start_time = timeit.default_timer()

    # Get the nearest neighbors
    distances4, indices4 = nn3.kneighbors(df_input)

    end_time = timeit.default_timer()
    execution_time = end_time - start_time
    print(f"ivfflat Execution time: {execution_time} seconds")

    return distances4, indices4, execution_time

distances4, indices4, execution_time_ivfflat = ivfflatSearchAlgorithm(df)

In [ ]:
print(indices3.head() == indices4.head())
print(indices4)

In [ ]:
import torch
import pointops

def knn_query_pointopsv1(n_neighbors_input, point_cloud_input, n_points_input):
    n_points_tensor = torch.cuda.FloatTensor([n_points_input])

    start_time = timeit.default_timer()

    reference_index, _ = pointops.knn_query(n_neighbors_input, point_cloud_input, n_points_tensor)

    torch.cuda.synchronize()

    reference_index_cpu = reference_index.cpu()

    # print(reference_index)

    end_time = timeit.default_timer()
    execution_time = end_time - start_time
    print(f"C++ PointopsV1 KNN Execution time: {execution_time} seconds")
    # print(reference_index.shape)

    return reference_index_cpu, execution_time

reference_index_cpu, execution_time_pointopsv1 = knn_query_pointopsv1(n_neighbors, point_cloud, n_points)

In [ ]:
print(type(indices1))
print(type(reference_index_cpu))

# Assuming you have a cuDF DataFrame (cudf_df) and a PyTorch Tensor (torch_tensor)
# Extract the data from the cuDF DataFrame
indices1_data = indices1.to_pandas().values

# Convert the cuDF data to a PyTorch Tensor
torch_indices1_data = torch.tensor(indices1_data).to(torch.int32)

# Compare the content element-wise
are_equal = torch.equal(torch_indices1_data, reference_index_cpu)

if are_equal:
    print("The content of the cuDF DataFrame and PyTorch Tensor is equal.")
else:
    print("The content differs between the cuDF DataFrame and PyTorch Tensor.")

In [ ]:
# Compare element-wise equality
elementwise_equal = torch.eq(torch_indices1_data, reference_index_cpu)

# Find indices where elements are different (i.e., where elementwise_equal is False)
differing_indices = torch.nonzero(~elementwise_equal)


print(differing_indices)
print(torch_indices1_data)
print(reference_index_cpu)

In [ ]:
import torch
from pointops2 import pointops2 as newerPointOps2

def knn_query_pointopsv2(n_neighbors_input, point_cloud_input, n_points_input):
    n_points_tensor = torch.cuda.FloatTensor([n_points_input])

    start_time = timeit.default_timer()

    reference_index, _ = newerPointOps2.knnquery(n_neighbors_input, point_cloud_input, None, n_points_tensor, n_points_tensor)

    torch.cuda.synchronize()

    reference_index_cpu2 = reference_index.cpu()

    end_time = timeit.default_timer()
    execution_time = end_time - start_time
    print(f"C++ PointopsV2 KNN Execution time: {execution_time} seconds")
    # print(reference_index)

    return reference_index_cpu2, execution_time

reference_index_cpu2, execution_time_pointopsv2 = knn_query_pointopsv2(n_neighbors, point_cloud, n_points)

In [ ]:
print(type(indices1))
print(type(reference_index_cpu2))

# Assuming you have a cuDF DataFrame (cudf_df) and a PyTorch Tensor (torch_tensor)
# Extract the data from the cuDF DataFrame
indices1_data = indices1.to_pandas().values

# Convert the cuDF data to a PyTorch Tensor
torch_indices1_data = torch.tensor(indices1_data).to(torch.int32)

# Compare the content element-wise
are_equal = torch.equal(torch_indices1_data, reference_index_cpu2)

if are_equal:
    print("The content of the cuDF DataFrame and PyTorch Tensor is equal.")
else:
    print("The content differs between the cuDF DataFrame and PyTorch Tensor.")

In [ ]:
# Compare element-wise equality
elementwise_equal = torch.eq(torch_indices1_data, reference_index_cpu2)

# Find indices where elements are different (i.e., where elementwise_equal is False)
differing_indices2 = torch.nonzero(~elementwise_equal)


print(differing_indices2)
print(torch_indices1_data)
print(reference_index_cpu2)

In [ ]:
torch.equal(differing_indices2, differing_indices)

In [ ]:
# # Create a cuML NearestNeighbors model
# nn4 = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ivfpq')
# # Compute the Dask DataFrame
# ddf_computed = ddf.compute()

# start_time = timeit.default_timer()



# # Fit the model with the input data
# nn4.fit(ddf_computed)

# # Get the nearest neighbors
# distances, indices = nn4.kneighbors(ddf_computed)

# end_time = timeit.default_timer()
# execution_time = end_time - start_time
# print(f"ivfflat Execution time: {execution_time} seconds")

In [ ]:
import matplotlib.pyplot as plt

# Execution times of each function
execution_times = [execution_time_auto, execution_time_rbc, execution_time_ivfflat, execution_time_pointopsv1, execution_time_pointopsv2]

# Function names
function_names = ['autoSearchAlgorithm', 'rbcSearchAlgorithm', 'ivfflatSearchAlgorithm', 'knn_query_pointopsv1', 'knn_query_pointopsv2']

# Number of n_points
n_points_used = n_points

# Plotting the execution times
plt.figure(figsize=(10, 6))
plt.bar(function_names, execution_times)
plt.xlabel('Function')
plt.ylabel('Execution Time (seconds)')
plt.title(f'Execution Time of Functions (n_points = {n_points_used})')
plt.xticks(rotation=45)
plt.show()


In [ ]:
import pandas as pd
import torch
import matplotlib.pyplot as plt

# List of n_points values
n_points_list = [500000, 1000000, 1500000, 2000000, 2500000, 3000000, 3500000, 4000000, 4500000, 5000000, 5500000] # Do not use a number above 5500000 if your GPU Memory is 16GB

# Lists to store execution times for each function
execution_times_auto = []
execution_times_rbc = []
execution_times_ivfflat = []
execution_times_pointopsv1 = []
execution_times_pointopsv2 = []

# Loop over n_points values
for n_points in n_points_list:

    print(f"Current n_points: {n_points}")

    # Generate random data for the DataFrame
    point_cloud = torch.cuda.FloatTensor(n_points, 3).uniform_()

    # Generate random data for the DataFrame
    random_data = np.array(point_cloud.cpu())

    # Create the cudf.DataFrame
    df = cudf.DataFrame(random_data, columns=["X", "Y", "Z"])

    # Call the functions and measure execution time
    distances_auto, indices_auto, execution_time_auto = autoSearchAlgorithm(df)
    distances_rbc, indices_rbc, execution_time_rbc = rbcSearchAlgorithm(df)
    distances_ivfflat, indices_ivfflat, execution_time_ivfflat = ivfflatSearchAlgorithm(df)
    reference_index_cpu, execution_time_pointopsv1 = knn_query_pointopsv1(n_neighbors, point_cloud, n_points)
    reference_index_cpu2, execution_time_pointopsv2 = knn_query_pointopsv2(n_neighbors, point_cloud, n_points)

    # Append execution times to the respective lists
    execution_times_auto.append(execution_time_auto)
    execution_times_rbc.append(execution_time_rbc)
    execution_times_ivfflat.append(execution_time_ivfflat)
    execution_times_pointopsv1.append(execution_time_pointopsv1)
    execution_times_pointopsv2.append(execution_time_pointopsv2)

# Create a DataFrame to store the execution times
execution_times_df = pd.DataFrame({
    'n_points': n_points_list,
    'autoSearchAlgorithm': execution_times_auto,
    'rbcSearchAlgorithm': execution_times_rbc,
    'ivfflatSearchAlgorithm': execution_times_ivfflat,
    'knn_query_pointopsv1': execution_times_pointopsv1,
    'knn_query_pointopsv2': execution_times_pointopsv2
})

# Plotting the execution times
plt.figure(figsize=(10, 6))
plt.plot(execution_times_df['n_points'], execution_times_df['autoSearchAlgorithm'], label='autoSearchAlgorithm')
plt.plot(execution_times_df['n_points'], execution_times_df['rbcSearchAlgorithm'], label='rbcSearchAlgorithm')
plt.plot(execution_times_df['n_points'], execution_times_df['ivfflatSearchAlgorithm'], label='ivfflatSearchAlgorithm')
plt.plot(execution_times_df['n_points'], execution_times_df['knn_query_pointopsv1'], label='knn_query_pointopsv1')
plt.plot(execution_times_df['n_points'], execution_times_df['knn_query_pointopsv2'], label='knn_query_pointopsv2')
plt.xlabel('n_points')
plt.ylabel('Execution Time (seconds)')
plt.title('Execution Time Comparison for Different n_points')
plt.legend()
plt.show()


In [ ]:
execution_times_df.to_csv('execution_times.csv', index=False)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
execution_times_df = pd.read_csv('execution_times.csv')

# Plotting the execution times
plt.figure(figsize=(10, 6))
plt.plot(execution_times_df['n_points'], execution_times_df['autoSearchAlgorithm'], label='autoSearchAlgorithm')
plt.plot(execution_times_df['n_points'], execution_times_df['rbcSearchAlgorithm'], label='rbcSearchAlgorithm')
plt.plot(execution_times_df['n_points'], execution_times_df['ivfflatSearchAlgorithm'], label='ivfflatSearchAlgorithm')
plt.plot(execution_times_df['n_points'], execution_times_df['knn_query_pointopsv1'], label='knn_query_pointopsv1')
plt.plot(execution_times_df['n_points'], execution_times_df['knn_query_pointopsv2'], label='knn_query_pointopsv2')
plt.xlabel('n_points')
plt.ylabel('Execution Time (seconds)')
plt.title('Execution Time Comparison for Different n_points')
plt.legend()
plt.show()
